In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.13.1


In [3]:
x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
result = session.run(f)
print(result)
session.close()

42


In [5]:
with tf.Session() as session:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
init = tf.global_variables_initializer()

with tf.Session() as session:
    init.run()
    result = f.eval()
    print(result)

42


In [7]:
session = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
session.close()

42


In [8]:
x1 = tf.Variable(1)

In [9]:
x1.graph

In [10]:
x2 = tf.Variable(2)

In [11]:
x2.graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
tf.reset_default_graph()

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [17]:
with tf.Session() as session:
    print(y.eval()) #10
    print(z.eval()) #15

10
15


In [18]:
# evaluate w and x once
with tf.Session() as session:
    y_val, z_val = session.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # theta = (XT * X)^(-1) * XT * y

with tf.Session() as session:
    theta_value = theta.eval()

In [20]:
theta_value

array([[-3.6845909e+01],
       [ 4.3667579e-01],
       [ 9.4581824e-03],
       [-1.0708507e-01],
       [ 6.4334780e-01],
       [-3.9223705e-06],
       [-3.7883162e-03],
       [-4.2040566e-01],
       [-4.3347809e-01]], dtype=float32)

In [21]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

standard_scaler = StandardScaler()
scaled_housing_data_plus_bias = standard_scaler.fit_transform(housing_data_plus_bias)

In [22]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  5.8034987
Epoch 100 MSE =  4.8678236
Epoch 200 MSE =  4.844326
Epoch 300 MSE =  4.8333654
Epoch 400 MSE =  4.82555
Epoch 500 MSE =  4.819826
Epoch 600 MSE =  4.815616
Epoch 700 MSE =  4.8125114
Epoch 800 MSE =  4.810215
Epoch 900 MSE =  4.8085113
Wall time: 689 ms


In [23]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  13.636479
Epoch 100 MSE =  5.038013
Epoch 200 MSE =  4.925084
Epoch 300 MSE =  4.8903728
Epoch 400 MSE =  4.86614
Epoch 500 MSE =  4.848669
Epoch 600 MSE =  4.8360586
Epoch 700 MSE =  4.826955
Epoch 800 MSE =  4.820381
Epoch 900 MSE =  4.815635
Wall time: 416 ms


In [ ]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)

init = tf.global_variables_initializer()

Wall time: 8.98 ms


In [ ]:
from tensorflow.python.client import device_lib

In [ ]:
def get_available_gpu_list():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [ ]:
get_available_gpu_list()

['/device:GPU:0', '/device:GPU:1']

In [ ]:
import pandas as pd

In [ ]:
df_usdcad = pd.read_csv("D:/Data/Forex/USDCAD.csv")
df_eurusd = pd.read_csv("D:/Data/Forex/EURUSD.csv")

In [ ]:
df_usdcad.describe()

,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume
count,7.227407e+07,7.227407e+07,7.227407e+07,7.227407e+07,7.227407e+07,7.227407e+07,7.227407e+07,7.227407e+07
mean,1.205706e+00,1.205719e+00,1.205694e+00,1.205706e+00,5.735902e+00,5.735728e+00,2.852410e+00,2.877631e+00
std,1.384800e-01,1.384778e-01,1.384822e-01,1.384799e-01,6.733740e+00,6.733505e+00,3.872871e+00,3.912291e+00
min,9.407100e-01,9.407500e-01,9.406800e-01,9.406800e-01,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.044810e+00,1.044830e+00,1.044790e+00,1.044810e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00
50%,1.271650e+00,1.271660e+00,1.271640e+00,1.271650e+00,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00
75%,1.316450e+00,1.316460e+00,1.316440e+00,1.316450e+00,7.000000e+00,7.000000e+00,4.000000e+00,4.000000e+00
max,1.469020e+00,1.469020e+00,1.469010e+00,1.469020e+00,1.553000e+03,1.553000e+03,7.760000e+02,1.009000e+03


In [ ]:
df_eurusd.describe()

,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume
count,8.233232e+07,8.233232e+07,8.233232e+07,8.233232e+07,8.233232e+07,8.233232e+07,8.233232e+07,8.233232e+07
mean,1.229096e+00,1.229111e+00,1.229082e+00,1.229096e+00,6.966572e+00,6.966475e+00,3.411229e+00,3.548147e+00
std,1.242919e-01,1.242968e-01,1.242870e-01,1.242919e-01,8.156638e+00,8.156578e+00,4.567997e+00,4.780315e+00
min,1.034080e+00,1.034140e+00,1.034040e+00,1.034080e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.124920e+00,1.124930e+00,1.124900e+00,1.124920e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00
50%,1.195420e+00,1.195430e+00,1.195420e+00,1.195420e+00,4.000000e+00,4.000000e+00,2.000000e+00,2.000000e+00
75%,1.326300e+00,1.326320e+00,1.326290e+00,1.326300e+00,9.000000e+00,9.000000e+00,4.000000e+00,5.000000e+00
max,1.603880e+00,1.603900e+00,1.603840e+00,1.603850e+00,1.099000e+03,1.099000e+03,5.440000e+02,5.450000e+02


In [ ]:
df_usdcad.shape

(72274072, 10)

In [ ]:
df_eurusd.shape

(82332324, 10)

In [ ]:
df_eurusd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332324 entries, 0 to 82332323
Data columns (total 10 columns):
Date               object
 Time              object
 Open              float64
 High              float64
 Low               float64
 Last              float64
 Volume            int64
 NumberOfTrades    int64
 BidVolume         int64
 AskVolume         int64
dtypes: float64(4), int64(4), object(2)
memory usage: 6.1+ GB


In [ ]:
df_usdcad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72274072 entries, 0 to 72274071
Data columns (total 10 columns):
Date               object
 Time              object
 Open              float64
 High              float64
 Low               float64
 Last              float64
 Volume            int64
 NumberOfTrades    int64
 BidVolume         int64
 AskVolume         int64
dtypes: float64(4), int64(4), object(2)
memory usage: 5.4+ GB


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_usdcad.rename(columns=lambda x: x.strip(), inplace=True)
df_eurusd.rename(columns=lambda x: x.strip(), inplace=True)

In [ ]:
df_usdcad["DateTime"] = df_usdcad["Date"].map(str) + " " + df_usdcad["Time"]
df_eurusd["DateTime"] = df_eurusd["Date"].map(str) + " " + df_eurusd["Time"]

In [ ]:
df_usdcad.head()

,Date,Time,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume,DateTime
0,2007/12/3,09:17:13,1.00261,1.00264,1.00261,1.00264,2,2,0,0,2007/12/3 09:17:13
1,2007/12/3,09:17:21,1.00258,1.00258,1.00249,1.00250,3,3,0,0,2007/12/3 09:17:21
2,2007/12/3,09:17:24,1.00264,1.00264,1.00251,1.00251,3,3,0,0,2007/12/3 09:17:24
3,2007/12/3,09:17:28,1.00250,1.00264,1.00250,1.00264,5,5,0,0,2007/12/3 09:17:28
4,2007/12/3,09:17:35,1.00267,1.00267,1.00267,1.00267,1,1,0,0,2007/12/3 09:17:35


In [ ]:
df_eurusd.head()

,Date,Time,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume,DateTime
0,2007/12/3,09:16:54,1.46351,1.46354,1.46351,1.46354,2,2,0,0,2007/12/3 09:16:54
1,2007/12/3,09:16:56,1.46351,1.46351,1.46347,1.46347,3,3,0,0,2007/12/3 09:16:56
2,2007/12/3,09:17:01,1.46354,1.46354,1.46354,1.46354,1,1,0,0,2007/12/3 09:17:01
3,2007/12/3,09:17:04,1.46346,1.46347,1.46344,1.46344,3,3,0,0,2007/12/3 09:17:04
4,2007/12/3,09:17:08,1.46347,1.46347,1.46338,1.46344,5,5,0,0,2007/12/3 09:17:08


In [ ]:
df_usdcad['DateTime'] = pd.to_datetime(df_usdcad['DateTime'])

In [ ]:
df_eurusd['DateTime'] = pd.to_datetime(df_eurusd['DateTime'])

In [ ]:
plt.scatter(df_usdcad["DateTime"], df_usdcad["Last"])

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001FBB350EA60> (for post_execute):


KeyboardInterrupt: 